<img width="8%" alt="HubSpot.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Send contacts to Google Sheets spreadsheet
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/HubSpot/HubSpot_Send_contacts_to_gsheet.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Send+contacts+to+gsheet:+Error+short+description">Bug report</a>

**Tags:** #hubspot #crm #sales #contact #naas_drivers #gsheet #automation #googlesheets

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-12-04 (Created: 2022-02-21)

**Description:** This notebook allows you to send HubSpot contacts to a Google Sheets spreadsheet sheet and get update your data updated everyday.

## Input

### Import libraries

In [ ]:
from naas_drivers import hubspot, gsheet
import naas
from datetime import date
import naas_data_product
import requests
import pandas as pd

### Setup variables
**Pre-requisite**

[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)

**Mandatory**
- `hs_access_token`: This is your HubSpot access token. It's used to authenticate your application's requests to the HubSpot API.
- `contact_properties`: List of properties (hubspot internal names) you want to get from contact. By default, you will get: email, firstname, lastname, createdate, lastmodifieddate, hs_object_id. To list of contact properties, you can use this template: "HubSpot/HubSpot_List_contact_properties.ipynb" stored in https://github.com/jupyter-naas/awesome-notebooks.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.

In [ ]:
# Inputs
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
contact_properties = [
    "firstname",
    "lastname",
    'email',
    'linkedinbio',
    'company',
    'createdate',
    "hs_object_id",
    "hubspot_owner_id",
]

# Outputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET")
sheet_name = "CONTACTS"
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
output_file = "crm_contacts"

## Model

### Get all contacts

In [ ]:
df_init = pload(output_dir, output_file)
if df_init is None:
    df_init = hubspot.connect(hs_access_token).contacts.get_all(contact_properties)

print("CRM Contacts:", len(df_init))
df_init.head(5)

### Get owners

In [ ]:
def get_owners(hs_access_token):
    # Request
    headers = {
        'accept': "application/json",
        'content-type': "application/json",
        'authorization': f"Bearer {hs_access_token}"
    }
    url = "https://api.hubapi.com/crm/v3/owners"

    # Response
    res = requests.get(url, headers=headers)
    return pd.DataFrame(res.json()['results'])

df_owners = get_owners(hs_access_token)
df_owners

### Prep data

In [ ]:
def prep_data(
    df_init,
    contact_properties,
    df_owners
):
    # Init
    df = df_init.copy()
    
    # Cleaning
    df = df[contact_properties]
    df.columns = df.columns.str.upper()
    to_rename = {
        "LINKEDINBIO": "LINKEDIN_URL",
        "HUBSPOT_OWNER_ID": "OWNER_ID",
        "HS_OBJECT_ID": "CONTACT_ID",
    }
    df = df.rename(columns=to_rename)
    df = df.astype(str)
    df = df.replace("None", "NA").replace("TBD", "NA")
    
    # Enrich
    owners = df_owners.set_index('id')['email'].to_dict()
    df["OWNER_EMAIL"] = df["OWNER_ID"].map(owners).fillna("NA")
    df.insert(loc=2, column="FULLNAME", value=df["FIRSTNAME"] + " " + df["LASTNAME"])
    
    # Display
    df = df.sort_values(by="CREATEDATE", ascending=False)
    return df.reset_index(drop=True)
    
df_clean = prep_data(df_init, contact_properties, df_owners)
print("CRM Contacts:", len(df_clean))
df_clean.head(10)

## Output

### Save data

In [ ]:
pdump(output_dir, df_clean, output_file)

### Send contacts to gsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(
    sheet_name=sheet_name,
    data=df_clean,
    append=False
)